In [129]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp

In [130]:
df = pd.read_csv('data.csv', index_col='Run')
df['molF [mmol/min]'] = df['VolF [mL/min]'] /0.08206/(273.15+25)

In [131]:
df['Conv'] = (1-df['xNH3'])/(1+df['xNH3'])

In [132]:
df['molF_NH3'] = df['molF [mmol/min]'] * (1-df['Conv'])
df['molF_N2'] = df['molF [mmol/min]'] * df['Conv'] * 0.5
df['molF_H2'] = df['molF [mmol/min]'] * df['Conv'] * 1.5

In [133]:
df['xNH3_cal'] = df['molF_NH3'] / (df['molF_NH3'] + df['molF_N2'] + df['molF_H2'])
df['xN2_cal'] = df['molF_N2'] / (df['molF_NH3'] + df['molF_N2'] + df['molF_H2'])
df['xH2_cal'] = df['molF_H2'] / (df['molF_NH3'] + df['molF_N2'] + df['molF_H2'])

In [134]:
df['Fin [mmol/min]'] = df['molF [mmol/min]']
df['Fout [mmol/min]'] = df['molF_NH3'] + df['molF_N2'] + df['molF_H2']

In [135]:
df['Vin [mL/min]'] = df['Fin [mmol/min]'] * 0.08206 * (273.15+df['Temp [degC]'])
df['Vout [mL/min]'] = df['Fout [mmol/min]'] * 0.08206 * (273.15+df['Temp [degC]'])
df['Vavg [mL/min]'] = (df['Vout [mL/min]'] - df['Vin [mL/min]'])/np.log(df['Vout [mL/min]']/df['Vin [mL/min]'])

In [136]:
rb_cat = 8.2/11 # g/mL
rt_cat = 6.51 # g/mL
por = 1 - rb_cat/rt_cat
df['Vcat [mL]'] = df['Wcat [g]']/rb_cat
df['rt [min]'] = df['Vcat [mL]']*por/df['Vavg [mL/min]']

In [137]:
df.to_csv('data_treated.csv')

In [138]:
df_ref = df[df['Temp [degC]'] == 450]

In [139]:
df1 = pd.DataFrame()
df1['Temp [K]'] = df_ref['Temp [degC]'] + 273.15
df1['Wcat [g]'] = df_ref['Wcat [g]']
df1['Fin [mmol/min]'] = df_ref['Fin [mmol/min]']
df1['RT [min]'] = df_ref['rt [min]']
df1['Conv'] = df_ref['Conv']
df1.head(5)

,Temp [K],Wcat [g],Fin [mmol/min],RT [min],Conv
Run,,,,,
9,723.15,0.3,1.226182,0.003756,0.663894
10,723.15,0.3,2.452364,0.002028,0.441961
11,723.15,0.3,3.678546,0.001409,0.333333
